In [1]:
import numpy as np

In [20]:
class FFNN:
    def __init__(
            self,
            n_in,
            hidden_nodes: list,
            n_out: int = 1,
            use_bias: bool = True,
            Ws=None):
        '''Params
        - hidden_nodes: list with n nodes per layer
        '''
        self.n_in = n_in
        self.hidden_nodes = np.array(hidden_nodes)
        self.use_bias = use_bias
        self.n_out = n_out
        self.Ws = Ws if Ws is not None else self._init_weights()
        
    def get_state(self):
        state = {
            'n_in': self.n_in,
            'hidden_nodes': self.hidden_nodes,
            'n_out': self.n_out,
            'use_bias': self.use_bias,
            'Ws': self.Ws}
        return state
        
    def _init_weights(self):
        n_in = self.n_in
        if self.use_bias:
            n_in += 1
        W1 = np.random.normal(size=(n_in, self.hidden_nodes[0]))
        Ws = [W1]
        if len(self.hidden_nodes) > 1:
            for i, n_nodes in enumerate(self.hidden_nodes[1:], 1):
                n_in = self.hidden_nodes[i - 1]
                if self.use_bias:
                    n_in += 1
                W = np.random.normal(size=(n_in, n_nodes))
                Ws.append(W)
        # output
        n_in = self.hidden_nodes[-1]
        if self.use_bias:
            n_in += 1
        W = np.random.normal(size=(n_in, self.n_out))
        Ws.append(W)
        return Ws
        
    def forward_pass(self, X, activation, activation_out):
        activation = {'relu': self._relu}[activation]
        activation_out = {'sigmoid': self._sigmoid}[activation_out]
        X = np.array(X)
        if len(X.shape) == 1:
            X = np.array([X])
        for W in self.Ws[:-1]:
            if self.use_bias:
                X = self._append_bias(X)
            X = X @ W
            X = activation(X)
        # output layer
        if self.use_bias:
            X = self._append_bias(X)
        X = X @ self.Ws[-1]
        X = activation_out(X)
        return X
            
    def _append_bias(self, X):
        X = self._append_col(X, np.ones, to_front=True)
        return X
    
    @staticmethod
    def _append_col(X, col_func, to_front=False, **kwargs):
        n_rows = X.shape[0]
        if col_func == np.random.normal:
            new_col = col_func(size=(n_rows, 1), **kwargs)
        else:
            new_col = col_func(shape=(n_rows, 1), **kwargs)
        if to_front:
            X = np.concatenate([new_col, X], axis=1)
        else:
            X = np.concatenate([X, new_col], axis=1)
        return X
    
    @staticmethod
    def _relu(X):
        return(X.clip(0, None))
    
    @staticmethod
    def _sigmoid(X):
        return 1 / (1 + np.exp(-X))
    
    def mutate(self, scale: float):
        '''Randomly mutate weights
        Weights will be multiplied by ~N(1, scale)
        In early iterations, it can be large (say 0.5), but should
        gradually diminish as models start to settle
        Parameters:
        - scale: standard deviations
        '''
        for i, W in enumerate(self.Ws):
            noise = np.random.normal(loc=1, scale=scale, size=W.shape)
            W = np.multiply(W, noise)
            self.Ws[i] = W
            
    def add_input(self, scale=0.01):
        self.n_in += 1
        self.Ws[0] = self._append_row(self.Ws[0], np.random.normal, scale=scale)
        
    @staticmethod
    def _append_row(X, row_func, **kwargs):
        n_cols = X.shape[1]
        new_row = row_func(size=(1, n_cols), **kwargs)
        X = np.concatenate([X, new_row])
        return X        
    
    def add_node(self, scale=0.01):
        # add to the first layer that is < n_in
        for i, W in enumerate(self.Ws):
            n_out = W.shape[1]
            if n_out < self.n_in:
                try:
                    print(f'Attempting to add node to W[{i}] ({n_out} -> {n_out + 1})')
                    self.Ws[i + 1] = self._append_row(self.Ws[i + 1], np.random.normal, scale=scale)
                    self.Ws[i] = self._append_col(W, np.random.normal, scale=scale)
                    print('Success.')
                except IndexError:
                    print(f'Initializing first node in new layer ({i + 1})')
                    if self.use_bias:
                        n_out += 1
                    #new_W = np.random.normal(size=(n_out, 1), scale=scale)
                    new_W = np.ones((n_out, 1))
                    new_W[0, 0] = 0.
                    self.Ws.append(new_W)
                return

In [21]:
nn = FFNN(3, [2, 2])
nn.Ws

[array([[-0.34495504, -0.83167971],
        [-0.10801804,  0.04104169],
        [-1.40613953,  0.69822816],
        [ 0.70534989,  0.86141259]]),
 array([[ 0.98736757, -0.72805366],
        [-0.61364689, -2.48442399],
        [-1.53047817,  1.10544067]]),
 array([[-0.1683029 ],
        [-1.45126332],
        [ 1.0060313 ]])]

In [22]:
x = [[1, 2, 3], [-1, 2, 3]]
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93382207],
       [0.9279524 ]])

In [23]:
nn.add_node()
nn.Ws

Attempting to add node to W[0] (2 -> 3)
Success.


[array([[-3.44955043e-01, -8.31679713e-01, -1.63200003e-02],
        [-1.08018039e-01,  4.10416884e-02,  1.43441885e-02],
        [-1.40613953e+00,  6.98228163e-01, -2.28016877e-03],
        [ 7.05349894e-01,  8.61412594e-01,  3.36761478e-04]]),
 array([[ 9.87367575e-01, -7.28053660e-01],
        [-6.13646894e-01, -2.48442399e+00],
        [-1.53047817e+00,  1.10544067e+00],
        [-1.19828862e-03, -6.57918425e-03]]),
 array([[-0.1683029 ],
        [-1.45126332],
        [ 1.0060313 ]])]

In [24]:
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93382207],
       [0.9279524 ]])

In [25]:
nn.add_node()
nn.Ws

Attempting to add node to W[1] (2 -> 3)
Success.


[array([[-3.44955043e-01, -8.31679713e-01, -1.63200003e-02],
        [-1.08018039e-01,  4.10416884e-02,  1.43441885e-02],
        [-1.40613953e+00,  6.98228163e-01, -2.28016877e-03],
        [ 7.05349894e-01,  8.61412594e-01,  3.36761478e-04]]),
 array([[ 9.87367575e-01, -7.28053660e-01, -2.33758875e-03],
        [-6.13646894e-01, -2.48442399e+00,  9.54027664e-03],
        [-1.53047817e+00,  1.10544067e+00,  1.15573252e-03],
        [-1.19828862e-03, -6.57918425e-03,  4.15641075e-03]]),
 array([[-0.1683029 ],
        [-1.45126332],
        [ 1.0060313 ],
        [-0.00478485]])]

In [26]:
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93382167],
       [0.927952  ]])

In [27]:
nn.add_node()
nn.Ws

Attempting to add node to W[2] (1 -> 2)
Initializing first node in new layer (3)


[array([[-3.44955043e-01, -8.31679713e-01, -1.63200003e-02],
        [-1.08018039e-01,  4.10416884e-02,  1.43441885e-02],
        [-1.40613953e+00,  6.98228163e-01, -2.28016877e-03],
        [ 7.05349894e-01,  8.61412594e-01,  3.36761478e-04]]),
 array([[ 9.87367575e-01, -7.28053660e-01, -2.33758875e-03],
        [-6.13646894e-01, -2.48442399e+00,  9.54027664e-03],
        [-1.53047817e+00,  1.10544067e+00,  1.15573252e-03],
        [-1.19828862e-03, -6.57918425e-03,  4.15641075e-03]]),
 array([[-0.1683029 ],
        [-1.45126332],
        [ 1.0060313 ],
        [-0.00478485]]),
 array([[0.],
        [1.]])]

In [28]:
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93382167],
       [0.927952  ]])

In [29]:
nn.add_node()
nn.Ws

Attempting to add node to W[2] (1 -> 2)
Success.


[array([[-3.44955043e-01, -8.31679713e-01, -1.63200003e-02],
        [-1.08018039e-01,  4.10416884e-02,  1.43441885e-02],
        [-1.40613953e+00,  6.98228163e-01, -2.28016877e-03],
        [ 7.05349894e-01,  8.61412594e-01,  3.36761478e-04]]),
 array([[ 9.87367575e-01, -7.28053660e-01, -2.33758875e-03],
        [-6.13646894e-01, -2.48442399e+00,  9.54027664e-03],
        [-1.53047817e+00,  1.10544067e+00,  1.15573252e-03],
        [-1.19828862e-03, -6.57918425e-03,  4.15641075e-03]]),
 array([[-1.68302900e-01, -2.11835213e-02],
        [-1.45126332e+00, -5.69087588e-03],
        [ 1.00603130e+00, -4.83338167e-03],
        [-4.78485142e-03, -8.81977099e-04]]),
 array([[0.        ],
        [1.        ],
        [0.00118664]])]

In [30]:
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93382167],
       [0.927952  ]])

In [31]:
nn.mutate(scale=0.3)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.95833738],
       [0.95480663]])

In [32]:
nn.mutate(scale=0.1)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.92474702],
       [0.91934798]])

In [33]:
nn.mutate(scale=0.01)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.92704201],
       [0.92195121]])

In [34]:
nn.add_input()
nn.Ws

[array([[-4.36480427e-01, -1.34598217e+00, -9.58369830e-03],
        [-6.57209079e-02,  1.43067879e-02,  1.67679944e-02],
        [-1.37095057e+00,  8.13573908e-01, -3.08791483e-03],
        [ 6.04613998e-01,  5.65159829e-01,  2.52687220e-04],
        [ 2.38981148e-02,  5.96990796e-03, -1.17718723e-02]]),
 array([[ 1.57962352e+00, -9.64489067e-01, -2.78299293e-03],
        [-6.89882315e-01, -2.80201054e+00,  7.88437376e-03],
        [-9.05896206e-01,  1.12625686e+00,  1.23598886e-03],
        [-5.39710066e-04, -4.37407241e-03,  4.10279230e-03]]),
 array([[-2.13169687e-01, -3.90672852e-02],
        [-1.21189603e+00, -6.39912406e-03],
        [ 1.37349601e+00, -2.31180189e-03],
        [-4.02652205e-03, -7.96666087e-04]]),
 array([[0.00000000e+00],
        [1.64866872e+00],
        [1.42233355e-03]])]

In [35]:
x = [[1, 2, 3, 4], [-1, 2, 3, 5]]
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93105657],
       [0.92725609]])

In [37]:
state = nn.get_state()
state

{'n_in': 4,
 'hidden_nodes': array([2, 2]),
 'n_out': 1,
 'use_bias': True,
 'Ws': [array([[-4.36480427e-01, -1.34598217e+00, -9.58369830e-03],
         [-6.57209079e-02,  1.43067879e-02,  1.67679944e-02],
         [-1.37095057e+00,  8.13573908e-01, -3.08791483e-03],
         [ 6.04613998e-01,  5.65159829e-01,  2.52687220e-04],
         [ 2.38981148e-02,  5.96990796e-03, -1.17718723e-02]]),
  array([[ 1.57962352e+00, -9.64489067e-01, -2.78299293e-03],
         [-6.89882315e-01, -2.80201054e+00,  7.88437376e-03],
         [-9.05896206e-01,  1.12625686e+00,  1.23598886e-03],
         [-5.39710066e-04, -4.37407241e-03,  4.10279230e-03]]),
  array([[-2.13169687e-01, -3.90672852e-02],
         [-1.21189603e+00, -6.39912406e-03],
         [ 1.37349601e+00, -2.31180189e-03],
         [-4.02652205e-03, -7.96666087e-04]]),
  array([[0.00000000e+00],
         [1.64866872e+00],
         [1.42233355e-03]])]}

In [39]:
new_nn = FFNN(**state)
y = new_nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.93105657],
       [0.92725609]])

In [40]:
import pandas as pd

In [43]:
metrics = pd.read_csv('../data/stock_metrics.csv')
metrics.head()

,stock,price,direction,RSI,RSIRev,fair_value_mult,geomean,sharpe,weighted_sharpe
0,AAON,91.150002,1,0.791933,0.208067,0.888403,0.985380,0.781409,0.899917
1,AAPL,169.300003,1,0.288738,0.711262,0.598491,0.097327,0.917493,1.257053
2,ABBV,159.619995,0,0.041134,0.958866,0.620334,0.355890,0.693304,0.724439
3,ABNB,164.229996,0,0.677844,0.322156,0.638398,0.928276,0.329321,0.588862
4,ACN,308.010010,0,0.005192,0.994808,0.382446,0.060568,0.741023,2.012313


In [47]:
x = metrics[['direction', 'RSI', 'fair_value_mult', 'geomean', 'sharpe', 'weighted_sharpe']]
x.shape

(221, 6)

In [50]:
x.values.shape

(221, 6)

In [51]:
test_nn =  FFNN(6, [6])

In [54]:
out = test_nn.forward_pass(x, 'relu', 'sigmoid')
out

array([[0.35852775],
       [0.36436748],
       [0.410528  ],
       [0.392903  ],
       [0.38822692],
       [0.36978286],
       [0.39824137],
       [0.42147935],
       [0.43325314],
       [0.3433566 ],
       [0.40930727],
       [0.40999065],
       [0.36186192],
       [0.38314643],
       [0.41457799],
       [0.36850939],
       [0.33684209],
       [0.39248488],
       [0.35322054],
       [0.38591879],
       [0.35373233],
       [0.405933  ],
       [0.35829199],
       [0.41774094],
       [0.36555753],
       [0.42912208],
       [0.41047905],
       [0.37861955],
       [0.39332644],
       [0.39629302],
       [0.47250278],
       [0.41027096],
       [0.38218519],
       [0.31885651],
       [0.40035127],
       [0.36369478],
       [0.37459269],
       [0.36062826],
       [0.41268142],
       [0.3297141 ],
       [0.37413017],
       [0.40164925],
       [0.4152147 ],
       [0.38459312],
       [0.37336816],
       [0.39127472],
       [0.3981432 ],
       [0.366

In [61]:
out.squeeze().shape

(221,)

In [62]:
pd.DataFrame({'stock': metrics.stock, 'value': out.squeeze()})

,stock,value
0,AAON,0.358528
1,AAPL,0.364367
2,ABBV,0.410528
3,ABNB,0.392903
4,ACN,0.388227
...,...,...
216,WMT,0.370657
217,XPEV,0.348675
218,YTRA,0.424787
219,ZBRA,0.373806
